In [19]:
import sys
import os

from acl_papers import crawler
from acl_papers import ACL_PAPER_PATH

PUB_YEAR = 2024
URLS = {
    'short': f'https://aclanthology.org/volumes/{PUB_YEAR}.eacl-short/',
    'long': f'https://aclanthology.org/volumes/{PUB_YEAR}.eacl-long/'
}
COLUMNS = ['length', 'title', 'abstract', 'pub_year', 'relevancy', 'related_domain']
ROWS = []
output_file = os.path.join(ACL_PAPER_PATH, 'acl_data.csv')


In [22]:
df = crawler.load_or_create_df(ACL_PAPER_PATH, columns=COLUMNS)

for length, url in URLS.items():
    soup = crawler.crawling(url=url)
    html_titles = soup.find_all('p', class_="d-sm-flex align-items-stretch")[1:]
    html_abstracts = soup.find_all('div', class_="card bg-light mb-2 mb-lg-3 collapse abstract-collapse")
    
    assert(len(html_titles) == len(html_abstracts))

    for html_title, html_abstract in zip(html_titles, html_abstracts):
        title = html_title.find_all(lambda tag: tag.name == 'a' and 'class' in tag.attrs and tag.attrs['class'] == ['align-middle'])
        assert(len(title) == 1)
        
        abstract = html_abstract.find_all(lambda tag: tag.name == 'div' and 'class' in tag.attrs and tag.attrs['class'] == ['card-body', 'p-3', 'small'])
        assert(len(abstract) == 1)
        row = dict()
               
        row['length'] = length
        row['title'] = title[0].text
        row['abstract'] = abstract[0].text
        row['pub_year'] = PUB_YEAR
        row['relevancy'] = None
        row['related_domain'] = None

        ROWS.append(row)

    
    


In [25]:

crawler.update_dataframe(path=output_file, columns=COLUMNS, new_rows=ROWS)